# Chirp Stimulus analysis 
Tobias Rose 2020

# Analysis

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
%matplotlib widget
import seaborn as sns
sns.set()  # set plot styles
import sys
import os
from pathlib import Path
from scipy.signal import convolve
from scipy.stats import zscore
from scipy.ndimage import gaussian_filter1d
# from scipy.stats import zscore ### Cave! scipy zscore does not handle NaN!
from ScanImageTiffReader import ScanImageTiffReader
from helpers import parse_SI_header as pSI #own
from tqdm import tqdm

import rastermap as rm
import pandas as pd


from sklearn.decomposition import NMF

# Function definitions

### Aux loading

In [3]:
def load_auxdata(filename):
    """ Loads .lvd aux data file """
    with open(filename, 'rb') as f:
        # Reset file index
        f.seek(0)
        # Get meta data
        samplingfreq = np.fromfile(f, dtype='>f8', count=1)
        print("Aux sampling frequency = {}Hz".format(samplingfreq))
        n_channels = int(np.fromfile(f, dtype='>f8', count=1))
        print("# channels = {}".format(n_channels))
        timestamp = np.fromfile(f, dtype='>f8', count=1)
        print("timestamp = {}".format(timestamp))
        max_input = np.fromfile(f, dtype='>f8', count=1)
        print("max input = {} V".format(max_input))
        # Read aux data
        auxdata = np.fromfile(f, dtype='>f8')
        n_datapoints = int(auxdata.shape[0]/n_channels)
        print("number of aux datapoints = {}".format(n_datapoints))
        auxdata = np.reshape(auxdata,(n_datapoints,n_channels))
        return auxdata, samplingfreq

### Frame extraction

In [4]:
def get_frame_times(auxdata, Frames_chan):
    """ extracts frame onset times """
    len_aux = len(auxdata)
    pos = np.argwhere(auxdata[0:,Frames_chan] > 0.75 * np.max(auxdata[range(0,len_aux),Frames_chan ])) # work on diff of indices rather than on raw diff to prevent multi-smaple detection in up/ downstrokes
    diffpos = np.argwhere(np.diff(pos[0:,0]) > 1)
    frame_times = pos[diffpos,0]
    
    if  len(frame_times)==0:
        print('get_frame_times WARNING: no frames found')
        frame_times = 1;
        return frame_times
    
    # find onset of first frame
    pos_first = np.argwhere(auxdata[0:,Frames_chan] < 0.5 * np.max(auxdata[range(0,len_aux), Frames_chan]))
    diffpos_first = np.argwhere(np.diff(pos_first[0:,0]) > 1)
    frame_times = np.append(diffpos_first[0], frame_times)
    
    return frame_times

### CHIRP - stimulus bound extraction

In [5]:
def get_stimIDs_chirp(auxdata, stimops):
    """ extracts chirp onset times """
    
    Frames_chan = stimops['Frames_chan']
    Stims_chan = stimops['Stims_chan']
    eye1_chan = stimops['eye1_chan']
    eye2_chan = stimops['eye2_chan']
    level = stimops['level']
    minsample_delta = stimops['minsample_delta']
    
    frame_times         = get_frame_times(auxdata,Frames_chan)
    frame_times_level   = frame_times[range(0,len(frame_times),level)]

    
    StimOn = auxdata[frame_times_level, Stims_chan]>0.8

    # generate cleaned eye binaries
    Eye1On = auxdata[frame_times_level,eye1_chan]*-1+ np.max(auxdata[frame_times_level,eye1_chan])>0.8
    Eye2On = auxdata[frame_times_level,eye2_chan]>0.8
    Eye2On[-1] = 1 
    bino = Eye1On == Eye2On

    Eye1On_only = Eye1On != bino
    Eye2On_only = Eye2On != bino
    Eye1On_only[-1] = False
    Eye2On_only[-1] = False
    bino[0] = False
    bino[-1] = False

    # generate cleaned bino binary
    bino_onsets_temp  = np.argwhere(np.diff(np.multiply(bino, 1)) > 0)
    bino_offsets_temp = np.argwhere(np.diff(np.multiply(bino, 1)) < 0)

    bino_onsets  = bino_onsets_temp[np.argwhere(bino_offsets_temp[0:,0] - bino_onsets_temp[0:,0] > minsample_delta)]
    bino_offsets = bino_offsets_temp[np.argwhere(bino_offsets_temp[0:,0] - bino_onsets_temp[0:,0] > minsample_delta)]                              

    bino_clean = np.full(( len(frame_times_level)), False) 

    for i in range(len(bino_onsets)):
        bino_clean[range(bino_onsets[i,0,0], bino_offsets[i,0,0])] = True

    # extract chirp stim on and offsets
    chirp_onsets_temp  = np.argwhere(np.diff(np.multiply(StimOn, 1)) > 0)
    chirp_offsets_temp = np.argwhere(np.diff(np.multiply(StimOn, 1)) < 0)

    chirp_on  = np.argwhere(np.diff(chirp_onsets_temp[0:,0]) > minsample_delta) + 1
    chirp_off = np.argwhere(np.diff(chirp_offsets_temp[0:,0]) > minsample_delta)

    chirp_on  = np.append(0, chirp_on)
    chirp_off = np.append(chirp_off, len(chirp_offsets_temp) - 1)

    chirp_onsets  = chirp_onsets_temp[chirp_on]
    chirp_offsets = chirp_offsets_temp[chirp_off]

    ids = { 
    'Contra':   [np.intersect1d(chirp_onsets, np.argwhere(Eye1On_only)), np.intersect1d(chirp_offsets, np.argwhere(Eye1On_only))],
    'Ipsi': [np.intersect1d(chirp_onsets, np.argwhere(Eye2On_only)), np.intersect1d(chirp_offsets, np.argwhere(Eye2On_only))],
    'Bino':   [np.intersect1d(chirp_onsets, np.argwhere(bino_clean)), np.intersect1d(chirp_offsets, np.argwhere(bino_clean))],
    'FrameTimes_level': frame_times_level,
    'FrameTimes':       frame_times,
    }
    
    return ids
    

### on-the-fly panda function definition of zscore that handles NaNs

In [6]:
def z_score(df): return (df-df.mean())/df.std(ddof=0)
# def m_mean(df): return df.mean()

# Batch processing

### User-specific folders

In [7]:
if sys.platform == "darwin":
    ### Mac
    main_root = '/Volumes/archive_bonhoeffer_group$/David Laubender/Data/imaging data/DL_191024_6/ImagingData/' #location of original data
    adata     = '/Volumes/archive_bonhoeffer_group$/David Laubender/adata' #location of saved analyzed data
    ftemp     = '/Users/trose/Data/temp' #fast disk (local ssd for s2p binary files) 
    ftiff     = '/Users/trose/Data/s2p_tiff' #fast disk folder for concatenated tiffs (if needed)
elif sys.platform == "win32":
#     main_root = 'I:/David Laubender/Data/imaging data/DL_191106_2/ImagingData' #location of original data
    main_root = 'I:/David Laubender/Data/imaging data/DL_191024_6/ImagingData' #location of original data    
    adata     = 'I:/David Laubender/adata' #location of saved analyzed data
    ftemp     = 'C:/temp/trose/suite2ptemp' #fast disk (local ssd for s2p binary files  
    ftiff     = 'C:/temp/trose/s2p_tiff' #fast disk folder for concatenated tiffs (if needed)

In [8]:
exp = ['62283', '62284', '62285', '62286', '62287', '62288', '62289', '62290']

In [9]:
exp = ['62336']

In [10]:
exp = ['62284']

In [11]:
# exp = ['62284']

In [12]:
adata_s2ppath = []
adata_tiffpath = []
aux_files = []
single_tiff_file =[]
experimentator = []
mouse = []
date = []
experiment = []

for val in exp:
    s2pdir = list(Path(main_root).rglob('suite2p_exp'+val+'/')) #recursive
    tifffile = list(Path(os.path.join(*Path(s2pdir[0]).parts[0:-3], 'ImagingData', Path(s2pdir[0]).parts[-2])).glob('exp'+val+'*.tif')) #recursive search over main_root    
    experimentator.append(Path(s2pdir[0]).parts[1])
    mouse.append(Path(s2pdir[0]).parts[4])
    date.append(Path(s2pdir[0]).parts[6])
    experiment.append(val)
    try:
        adata_s2ppath.append(os.path.join(s2pdir[0], 'suite2p', 'combined'))
        adata_tiffpath.append(os.path.dirname(s2pdir[0]))
        aux_files.append(*Path(os.path.join(*Path(s2pdir[0]).parts[0:-3], 'data', Path(s2pdir[0]).parts[-2])).glob('exp'+val+'*.lvd'))
        single_tiff_file.append(str(tifffile[0]))
    except:
        print(val + ' not found')

### parse first tiff of exp for imaging specs

In [13]:
with ScanImageTiffReader(single_tiff_file[0]) as reader:
    header = (reader.description(0))
    mov_dim = (reader.shape())
    
level = pSI.parse_SI_header_level(header)
zoom = pSI.parse_SI_header_zoom(header)
framerate = pSI.parse_SI_header_FrameRate(header)
channels = pSI.parse_SI_header_Channels(header)
volumes = pSI.parse_SI_header_Volumes(header)
frames = pSI.parse_SI_header_Frames(header)
frames_per_file = pSI.parse_SI_header_FramesPerFile(header)

# account for multilevel acq where frames is 1
if frames < volumes:
    frames = volumes

### extract chirp stim timebase

In [14]:
aux_filename = str(aux_files[0])

In [15]:
[auxdata, aux_samplingfreq] = load_auxdata(aux_filename)

Aux sampling frequency = [5000.]Hz
# channels = 19
timestamp = [2.01912011e+13]
max input = [10.] V
number of aux datapoints = 6256640


In [16]:
stimops = {
    'Frames_chan': 3,
    'Stims_chan': 7,
    'eye1_chan': 16,
    'eye2_chan': 17,
    'level': level, # extract from SI file in the future
    'minsample_delta': 100 
     }

In [17]:
ids = get_stimIDs_chirp(auxdata, stimops)

In [18]:
# recalculate framerate based on auxdata
framerate_auxderived = aux_samplingfreq / np.median(np.diff(ids['FrameTimes']))


### plot aux_data

#### raw aux plus stimbounds

In [ ]:
fig = plt.figure(figsize=(8,6))
plt.subplot(3,1,1)
plt.plot(auxdata[ids['FrameTimes_level'],stimops['Stims_chan']]), plt.ylabel('Stims_chan') 
plt.vlines(ids['Ipsi'][0],-1,6, 'r')
plt.vlines(ids['Ipsi'][1],-1,6, 'r')
plt.vlines(ids['Contra'][0],-1,6, 'b')
plt.vlines(ids['Contra'][1],-1,6, 'b')
plt.vlines(ids['Bino'][0],-1,6, 'k')
plt.vlines(ids['Bino'][1],-1,6, 'k')
plt.subplot(3,1,2)
plt.plot(auxdata[ids['FrameTimes_level'],stimops['eye1_chan']]), plt.ylabel('eye1_chan') 
plt.subplot(3,1,3)
plt.plot(auxdata[ids['FrameTimes_level'],stimops['eye2_chan']]), plt.ylabel('eye2_chan') 
plt.show()

In [ ]:
fig = plt.figure(figsize=(8,6))
plt.subplot(3,1,1)
plt.plot(auxdata[:,stimops['Stims_chan']]), plt.ylabel('Stims_chan') 
plt.vlines(ids['FrameTimes_level'][ids['Ipsi'][0]],-1,6, 'r')
plt.vlines(ids['FrameTimes_level'][ids['Ipsi'][1]],-1,6, 'k')
# plt.vlines(ids['Ipsi'][1],-1,6, 'r')
# plt.vlines(ids['Contra'][0],-1,6, 'b')
# plt.vlines(ids['Contra'][1],-1,6, 'b')
# plt.vlines(ids['Bino'][0],-1,6, 'k')
# plt.vlines(ids['Bino'][1],-1,6, 'k')
# plt.subplot(3,1,2)
# plt.plot(auxdata[ids['FrameTimes_level'],stimops['eye1_chan']]), plt.ylabel('eye1_chan') 
# plt.subplot(3,1,3)
# plt.plot(auxdata[ids['FrameTimes_level'],stimops['eye2_chan']]), plt.ylabel('eye2_chan') 
plt.show

# make PSTH

### load traces

In [21]:
F = np.load(os.path.join(adata_s2ppath[0],'F.npy'))
Fneu = np.load(os.path.join(adata_s2ppath[0],'Fneu.npy'))
spks = np.load(os.path.join(adata_s2ppath[0],'spks.npy'))
stat = np.load(os.path.join(adata_s2ppath[0],'stat.npy'), allow_pickle=True)
ops = np.load(os.path.join(adata_s2ppath[0],'ops.npy'), allow_pickle=True).item()
iscell = np.load(os.path.join(adata_s2ppath[0],'iscell.npy'))

### drop non-cells

In [22]:
old_len = len(F)

F = F[iscell[:,0]==1,:]
Fneu =Fneu[iscell[:,0]==1,:]
spks = spks[iscell[:,0]==1,:]

print('dropped ' + str(old_len - len(F)) + ' non-boutons')

dropped 185 non-boutons


### PANDAS: generate seaborn-compatible, stimulus-chopped long-form pandas and plot (maybe not)

In [23]:
categories = ['Ipsi', 'Contra', 'Bino'] #ids.keys()

ncells = F.shape[0]

prestim  = 1 / (framerate_auxderived / level) * 10 #seconds before stimulus
poststim = 1 / (framerate_auxderived / level) * 10 #seconds after stimulus

prestim_frames = np.round(prestim * (framerate_auxderived / level))
poststim_frames = np.round(prestim * (framerate_auxderived / level))

In [24]:
dfs = []
dataframe = []
maxlength = np.empty(3)
# allocate PSTH array
maxlength[0] = np.int(np.ceil(np.diff(ids["Contra"][:], axis = 0).mean() + prestim_frames + poststim_frames))
maxlength[1] = np.int(np.ceil(np.diff(ids["Ipsi"][:], axis = 0).mean() + prestim_frames + poststim_frames))
maxlength[2] = np.int(np.ceil(np.diff(ids["Bino"][:], axis = 0).mean() + prestim_frames + poststim_frames))

maxlength = np.int(np.max(maxlength))
# for cell in tqdm(np.random.randint(0,ncells,5)):
for cell in tqdm(range(ncells)):
    for category in categories:
        for trial, val in enumerate(ids[category][0]):
            
            aligned_F = np.empty((1,maxlength)).squeeze()
            aligned_F.fill(np.NaN)
            aligned_spks = np.empty((1,maxlength)).squeeze()
            aligned_spks.fill(np.NaN)
            aligned_stimdata = np.empty((1,maxlength)).squeeze()
            aligned_stimdata.fill(np.NaN)
            
            slice_range = np.arange(ids[category][0][trial].astype(int)-prestim_frames.astype(int),ids[category][1][trial].astype(int)+poststim_frames.astype(int))
            
            aligned_stimdata[0:len(slice_range)] = auxdata[ids['FrameTimes_level'][slice_range],stimops['Stims_chan']]
#             aligned_stimdata = auxdata[ids['FrameTimes_level'][slice_range][0]:ids['FrameTimes_level'][slice_range][-1],stimops['Stims_chan']]
            
            aligned_F[0:len(slice_range)] = F[cell,slice_range]
            aligned_spks[0:len(slice_range)] = spks[cell,slice_range]
            
            time = np.arange(0, aligned_F.shape[0] * 1 / (framerate_auxderived / level), 1 / (framerate_auxderived / level) ) - prestim
            aux_time = np.arange(0, aligned_stimdata.shape[0] * 1 / (framerate_auxderived / level), 1 / (framerate_auxderived / level)  ) - prestim
            
            dfs.append(pd.DataFrame({
                                    'experimentator': experimentator[0],
                                    'mouse': mouse[0],
                                    'date': date[0],
                                    'experiment': experiment[0],
                                    'cell': cell,
                                    'time': time,
                                    'auxtime':aux_time,
                                    'aux_files': aux_files[0],
                                    'aligned_stimdata': aligned_stimdata,
                                    'trial':trial,
                                    'aligned_F': aligned_F,
                                    'aligned_spks': aligned_spks,
                                    'category': category,
                                    'adata_s2ppath': adata_s2ppath[0],
                                    'SI_level': level,
                                    'SI_zoom': zoom,
                                    'SI_framerate': framerate,
                                    'SI_framerate_aux_derived': framerate_auxderived[0],
                                    'SI_channels': channels,
                                    'SI_volumes': volumes,
                                    'SI_frames': frames,
                                    'SI_frames_per_file': frames_per_file,
                                    'SI_tiffpath': adata_tiffpath[0],
                                    'SI_single_tiff_file': single_tiff_file[0]
                                                    
                                    }))            
dataframe = pd.concat(dfs, axis=0)

# TODO
# - include Fneu, dF/F (-Fneu), reconvolved
# - functionalize
# - aggregate over all recordings

100%|████████████████████████████████████████████████████████████████████████████████| 741/741 [01:09<00:00, 10.59it/s]


### 'Simple' one-liners to get trial averages over all cells

In [ ]:

fig = plt.figure(figsize=(10,8))
plt.subplot(2,3,1)
dataframe.loc[dataframe.loc[:,'category'] == 'Ipsi'].groupby(['time', 'cell'], as_index=False).aligned_spks.mean().pivot(index = 'time', columns='cell').mean(axis = 1).plot()
plt.subplot(2,3,2)
dataframe.loc[dataframe.loc[:,'category'] == 'Contra'].groupby(['time', 'cell'], as_index=False).aligned_spks.mean().pivot(index = 'time', columns='cell').mean(axis = 1).plot()
plt.subplot(2,3,3)
dataframe.loc[dataframe.loc[:,'category'] == 'Bino'].groupby(['time', 'cell'], as_index=False).aligned_spks.mean().pivot(index = 'time', columns='cell').mean(axis = 1).plot()
plt.subplot(2,3,4)
dataframe.loc[dataframe.loc[:,'category'] == 'Ipsi'].groupby(['time', 'cell'], as_index=False).aligned_F.mean().pivot(index = 'time', columns='cell').mean(axis = 1).plot()
plt.subplot(2,3,5)
dataframe.loc[dataframe.loc[:,'category'] == 'Contra'].groupby(['time', 'cell'], as_index=False).aligned_F.mean().pivot(index = 'time', columns='cell').mean(axis = 1).plot()
plt.subplot(2,3,6)
dataframe.loc[dataframe.loc[:,'category'] == 'Bino'].groupby(['time', 'cell'], as_index=False).aligned_F.mean().pivot(index = 'time', columns='cell').mean(axis = 1).plot()


# generate sorted mean-cell responses / zscores or not

In [25]:
# generate sorted mean-cell responses / zscores or not

allcells_Ipsi_mean = dataframe.loc[dataframe.loc[:,'category'] == 'Ipsi'].groupby(['time', 'cell'], as_index=False).aligned_spks.mean().pivot(index = 'time', columns='cell')
allcells_Ipsi_mean_zscored = allcells_Ipsi_mean.transform((lambda x : z_score(x)))
sortidx = allcells_Ipsi_mean_zscored.max().sort_values(ascending=False).index
allcells_Ipsi_mean_sort_zscored = allcells_Ipsi_mean_zscored.reindex(sortidx, axis = 1)

allcells_Contra_mean = dataframe.loc[dataframe.loc[:,'category'] == 'Contra'].groupby(['time', 'cell'], as_index=False).aligned_spks.mean().pivot(index = 'time', columns='cell')
allcells_Contra_mean_zscored = allcells_Contra_mean.transform((lambda x : z_score(x)))
sortidx = allcells_Contra_mean_zscored.max().sort_values(ascending=False).index
allcells_Contra_mean_sort_zscored = allcells_Contra_mean_zscored.reindex(sortidx, axis = 1)

allcells_Bino_mean = dataframe.loc[dataframe.loc[:,'category'] == 'Bino'].groupby(['time', 'cell'], as_index=False).aligned_spks.mean().pivot(index = 'time', columns='cell')
allcells_Bino_mean_zscored = allcells_Bino_mean.transform((lambda x : z_score(x)))
sortidx = allcells_Bino_mean_zscored.max().sort_values(ascending=False).index
allcells_Bino_mean_sort_zscored = allcells_Bino_mean_zscored.reindex(sortidx, axis = 1)


allcells_Ipsi_mean = dataframe.loc[dataframe.loc[:,'category'] == 'Ipsi'].groupby(['time', 'cell'], as_index=False).aligned_spks.mean().pivot(index = 'time', columns='cell')
sortidx2 = allcells_Ipsi_mean.max().sort_values(ascending=False).index
allcells_Ipsi_mean_sort = allcells_Ipsi_mean.reindex(sortidx2, axis = 1)

allcells_Contra_mean = dataframe.loc[dataframe.loc[:,'category'] == 'Contra'].groupby(['time', 'cell'], as_index=False).aligned_spks.mean().pivot(index = 'time', columns='cell')
sortidx2 = allcells_Contra_mean.max().sort_values(ascending=False).index
allcells_Contra_mean_sort = allcells_Contra_mean.reindex(sortidx2, axis = 1)

allcells_Bino_mean = dataframe.loc[dataframe.loc[:,'category'] == 'Bino'].groupby(['time', 'cell'], as_index=False).aligned_spks.mean().pivot(index = 'time', columns='cell')
sortidx2 = allcells_Bino_mean.max().sort_values(ascending=False).index
allcells_Bino_mean_sort = allcells_Bino_mean.reindex(sortidx2, axis = 1)





# allcells_Ipsi_mean = dataframe.loc[dataframe.loc[:,'category'] == 'Ipsi'].groupby(['time', 'cell'], as_index=False).aligned_F.mean().pivot(index = 'time', columns='cell')
# allcells_Ipsi_mean_zscored = allcells_Ipsi_mean.transform((lambda x : z_score(x)))
# sortidx = allcells_Ipsi_mean_zscored.max().sort_values(ascending=False).index
# allcells_Ipsi_mean_sort_zscored = allcells_Ipsi_mean_zscored.reindex(sortidx, axis = 1)

# allcells_Contra_mean = dataframe.loc[dataframe.loc[:,'category'] == 'Contra'].groupby(['time', 'cell'], as_index=False).aligned_F.mean().pivot(index = 'time', columns='cell')
# allcells_Contra_mean_zscored = allcells_Contra_mean.transform((lambda x : z_score(x)))
# sortidx = allcells_Contra_mean_zscored.max().sort_values(ascending=False).index
# allcells_Contra_mean_sort_zscored = allcells_Contra_mean_zscored.reindex(sortidx, axis = 1)

# allcells_Bino_mean = dataframe.loc[dataframe.loc[:,'category'] == 'Bino'].groupby(['time', 'cell'], as_index=False).aligned_F.mean().pivot(index = 'time', columns='cell')
# allcells_Bino_mean_zscored = allcells_Bino_mean.transform((lambda x : z_score(x)))
# sortidx = allcells_Bino_mean_zscored.max().sort_values(ascending=False).index
# allcells_Bino_mean_sort_zscored = allcells_Bino_mean_zscored.reindex(sortidx, axis = 1)


# allcells_Ipsi_mean = dataframe.loc[dataframe.loc[:,'category'] == 'Ipsi'].groupby(['time', 'cell'], as_index=False).aligned_F.mean().pivot(index = 'time', columns='cell')
# sortidx2 = allcells_Ipsi_mean.max().sort_values(ascending=False).index
# allcells_Ipsi_mean_sort = allcells_Ipsi_mean.reindex(sortidx2, axis = 1)

# allcells_Contra_mean = dataframe.loc[dataframe.loc[:,'category'] == 'Contra'].groupby(['time', 'cell'], as_index=False).aligned_F.mean().pivot(index = 'time', columns='cell')
# sortidx2 = allcells_Contra_mean.max().sort_values(ascending=False).index
# allcells_Contra_mean_sort = allcells_Contra_mean.reindex(sortidx2, axis = 1)

# allcells_Bino_mean = dataframe.loc[dataframe.loc[:,'category'] == 'Bino'].groupby(['time', 'cell'], as_index=False).aligned_F.mean().pivot(index = 'time', columns='cell')
# allcells_Bino_mean_zscored = allcells_Bino_mean.transform((lambda x : z_score(x)))
# sortidx2 = allcells_Bino_mean.max().sort_values(ascending=False).index
# allcells_Bino_mean_sort = allcells_Bino_mean.reindex(sortidx2, axis = 1)

In [ ]:
# plot maps
fig = plt.figure(figsize=(12,6))

with sns.axes_style('white'):
    plt.subplot(2,3,1)
    plt.imshow(allcells_Ipsi_mean_sort_zscored.T, vmax = 3, vmin = -0.5, aspect = 'auto', cmap = 'Reds')
    plt.xlabel('time')
    plt.ylabel('bouton')
    plt.title('Ipsi')
    plt.vlines([prestim_frames, maxlength - poststim_frames],0,ncells-1, 'k')
    
    plt.subplot(2,3,2)
    plt.imshow(allcells_Contra_mean_sort_zscored.T, vmax = 3, vmin = -0.5, aspect = 'auto', cmap = 'Blues')
    plt.xlabel('time')
    plt.title('Contra')
    plt.vlines([prestim_frames, maxlength - poststim_frames],0,ncells-1, 'k')
        
    plt.subplot(2,3,3)
    plt.imshow(allcells_Bino_mean_sort_zscored.T, vmax = 3, vmin = -0.5, aspect = 'auto', cmap = 'gray_r')
    plt.xlabel('time [s]')
    plt.colorbar(label='zscore')
    plt.title('Bino')
    plt.vlines([prestim_frames, maxlength - poststim_frames],0,ncells-1, 'k')
    
    plt.subplot(2,3,4)
    plt.imshow(allcells_Ipsi_mean_sort.T, vmax = 4, vmin = -.5, aspect = 'auto', cmap = 'Reds')
    plt.xlabel('time')
    plt.ylabel('bouton')
    plt.title('Ipsi')
    plt.vlines([prestim_frames, maxlength - poststim_frames],0,ncells-1, 'k')
    
    plt.subplot(2,3,5)
    plt.imshow(allcells_Contra_mean_sort.T, vmax = 4, vmin = -.5, aspect = 'auto', cmap = 'Blues')
    plt.xlabel('time')
    plt.title('Contra')
    plt.vlines([prestim_frames, maxlength - poststim_frames],0,ncells-1, 'k')
    
    plt.subplot(2,3,6)
    plt.imshow(allcells_Bino_mean_sort.T, vmax = 4, vmin = -.5, aspect = 'auto', cmap = 'gray_r')
    plt.xlabel('time [s]')
    plt.title('Bino')
    plt.colorbar(label='deconvolved F [a.u.]')
    plt.vlines([prestim_frames, maxlength - poststim_frames],0,ncells-1, 'k')
    
    
    
#     plt.subplot(2,3,4)
#     plt.imshow(allcells_Ipsi_mean_sort.T, vmax = 150, vmin = 30, aspect = 'auto', cmap = 'Reds')
#     plt.xlabel('time')
#     plt.ylabel('bouton')
#     plt.title('Ipsi')
#     plt.vlines([prestim_frames, maxlength - poststim_frames],0,ncells-1, 'k')
    
#     plt.subplot(2,3,5)
#     plt.imshow(allcells_Contra_mean_sort.T, vmax = 150, vmin = 30, aspect = 'auto', cmap = 'Blues')
#     plt.xlabel('time')
#     plt.title('Contra')
#     plt.vlines([prestim_frames, maxlength - poststim_frames],0,ncells-1, 'k')
    
#     plt.subplot(2,3,6)
#     plt.imshow(allcells_Bino_mean_sort.T, vmax = 150, vmin = 30, aspect = 'auto', cmap = 'gray_r')
#     plt.xlabel('time [s]')
#     plt.title('Bino')
#     plt.colorbar(label='F')
#     plt.vlines([prestim_frames, maxlength - poststim_frames],0,ncells-1, 'k')
    
    plt.tight_layout()
    
    plt.show()

## Rastermap sortings

In [26]:
# all cells
sp_ipsi = allcells_Ipsi_mean_zscored.to_numpy()
sp_contra = allcells_Contra_mean_zscored.to_numpy()
sp_bino = allcells_Bino_mean_zscored.to_numpy()

sp_all = np.concatenate((allcells_Contra_mean.to_numpy(), allcells_Ipsi_mean.to_numpy(), allcells_Bino_mean.to_numpy()), axis = 0)

sp_ipsi[np.isnan(sp_ipsi)] = 0
sp_contra[np.isnan(sp_contra)] = 0
sp_bino[np.isnan(sp_bino)] = 0

sp_all[np.isnan(sp_all)] = 0

sp_all = zscore(sp_all) # needs to be done in numpy to be element-wise here - I collapsed the major categories. I dont think I can do that in pandas straight

In [27]:
%%capture
model_ipsi = rm.mapping.Rastermap(n_components=1).fit(sp_ipsi.T)
model_contra = rm.mapping.Rastermap(n_components=1).fit(sp_contra.T)
model_bino = rm.mapping.Rastermap(n_components=1).fit(sp_bino.T)
model_all = rm.mapping.Rastermap(n_components=1).fit(sp_all.T)

In [28]:
### sort neurons and smooth across neurons 
smooth = True
if smooth:
    isort = np.argsort(model_all.embedding[:,0])
    
    
    Sm_ipsi = gaussian_filter1d(sp_ipsi.T[isort,:], np.minimum(3,int(sp_ipsi.shape[0]*0.005)), axis=0)
    # isort = np.argsort(model_contra.embedding[:,0])
    Sm_contra = gaussian_filter1d(sp_contra.T[isort,:], np.minimum(3,int(sp_contra.shape[0]*0.005)), axis=0)
    # isort = np.argsort(model_bino.embedding[:,0])
    Sm_bino = gaussian_filter1d(sp_bino.T[isort,:], np.minimum(3,int(sp_bino.shape[0]*0.005)), axis=0)
    Sm_all  = gaussian_filter1d(sp_all.T[isort,:], np.minimum(3,int(sp_ipsi.shape[0]*0.005)), axis=0)
#     Sm_all= sp_all.T[isort,:]
else:
    isort = np.argsort(model_bino.embedding[:,0])
    Sm_ipsi = sp_ipsi.T[isort,:]
    # isort = np.argsort(model_contra.embedding[:,0])
    Sm_contra = sp_contra.T[isort,:]
    # isort = np.argsort(model_bino.embedding[:,0])
    Sm_bino = sp_bino.T[isort,:]
# (optional) smooth in time
# Sm = gaussian_filter1d(Sm, 1, axis=1)

### auxdata and timebase 

In [29]:
slice_range = np.arange(ids['Ipsi'][0][1].astype(int)-prestim_frames.astype(int),ids['Ipsi'][1][1].astype(int)+poststim_frames.astype(int)+1)
# slice_range = np.arange(ids[category][0][trial].astype(int)-prestim_frames.astype(int),ids[category][1][trial].astype(int)+poststim_frames.astype(int))

aligned_stimdata = auxdata[ids['FrameTimes_level'][slice_range],stimops['Stims_chan']]
# aligned_stimdata_concat = np.tile(auxdata[ids['FrameTimes_level'][slice_range],stimops['Stims_chan']], (0,3))
aux_time = np.arange(0, aligned_stimdata.shape[0] * 1 / (framerate_auxderived / level), 1 / (framerate_auxderived / level) ) - prestim

aligned_stimdata_full = auxdata[ids['FrameTimes_level'][slice_range[0]]:ids['FrameTimes_level'][slice_range[-1]],stimops['Stims_chan']]
aligned_stimdata_full_concat = np.tile(auxdata[ids['FrameTimes_level'][slice_range[0]]:ids['FrameTimes_level'][slice_range[-1]],stimops['Stims_chan']], (1,3)).T

aux_time_full = np.arange(0, aligned_stimdata_full.shape[0] * 1 / aux_samplingfreq , 1 / aux_samplingfreq   ) - prestim
aux_time_full_concat = np.arange(0, aligned_stimdata_full_concat.shape[0] * 1 / aux_samplingfreq , 1 / aux_samplingfreq   ) - prestim

plt.figure()
# plt.plot(aux_time, aligned_stimdata)
plt.plot(aux_time_full_concat, aligned_stimdata_full_concat)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### seperately sorted

In [130]:
### view neuron sorting

# aligned_stimdata = auxdata[ids['FrameTimes_level'][slice_range][0]:ids['FrameTimes_level'][slice_range][-1],stimops['Stims_chan']]
# aux_time = np.arange(0, aligned_stimdata.shape[0] * 1 / aux_samplingfreq, 1 / aux_samplingfreq  ) - prestim

fs = framerate_auxderived / level
vmin_glob = -.5
vmax_glob = 2.5

with sns.axes_style('white'):
    xl = [-prestim,Sm_contra.shape[1]/fs-prestim]
    
    fig8 = plt.figure(figsize=(16,6), constrained_layout=True)
    gs1 = fig8.add_gridspec(nrows=8, ncols=3)
    
    fig8.add_subplot(gs1[1:, :-2])      
    fig8_ax=plt.imshow(Sm_contra, vmin=vmin_glob,vmax=vmax_glob,aspect='auto',extent=[-prestim,Sm_contra.shape[1]/fs-prestim, 0,Sm_contra.shape[0]], cmap = 'Blues')
    plt.xlabel('time (s)', fontsize=18)    
    
    plt.vlines([prestim_frames / fs-prestim, maxlength / fs - poststim_frames / fs -prestim ],0,ncells-1, 'k')
    plt.ylabel('boutons', fontsize=18)     
        
    f8_ax2 = fig8.add_subplot(gs1[1:, 1:-1])
    ax=plt.imshow(Sm_ipsi, vmin=vmin_glob,vmax=vmax_glob,aspect='auto',extent=[-prestim,Sm_ipsi.shape[1]/fs-prestim, 0,Sm_ipsi.shape[0]], cmap = 'Reds')
    plt.xlabel('time (s)', fontsize=18)
    
    plt.vlines([prestim_frames / fs-prestim, maxlength / fs - poststim_frames / fs -prestim ],0,ncells-1, 'k')
        
    f8_ax3 = fig8.add_subplot(gs1[1:, 2:])
    ax=plt.imshow(Sm_bino, vmin=vmin_glob,vmax=vmax_glob,aspect='auto',extent=[-prestim,Sm_bino.shape[1]/fs-prestim, 0,Sm_bino.shape[0]], cmap = 'gray_r')
    plt.xlabel('time (s)', fontsize=18)
    
    plt.vlines([prestim_frames / fs-prestim, maxlength / fs - poststim_frames / fs -prestim],0,ncells-1, 'k')

#    plt.colorbar(label='deconvolved F [a.u.]')
   
    f8_ax4 = fig8.add_subplot(gs1[:1, :-2])    
    plt.plot(aux_time_full, aligned_stimdata_full, 'k')
#     plt.vlines([prestim_frames / fs-prestim, maxlength / fs - poststim_frames / fs -prestim],0,4, 'k')
    f8_ax4.set_xlim(*xl)
    f8_ax4.axis('off')
    plt.title('Contra')
    
    f8_ax5 = fig8.add_subplot(gs1[:1, 1:-1])
    plt.plot(aux_time_full, aligned_stimdata_full, 'k')
    f8_ax5.set_xlim(*xl)
    f8_ax5.axis('off')
    plt.title('Ipsi')

    
    f8_ax6 = fig8.add_subplot(gs1[:1, 2:])
    plt.plot(aux_time_full, aligned_stimdata_full, 'k')
    f8_ax6.set_xlim(*xl)
    f8_ax6.axis('off')
    plt.title('Bino')    
    
#     plt.tight_layout()           
    plt.suptitle('z-scored deconvolved Chirp PSTHs - rastermap embedding (sorted on Bino, smoothened over boutons) - exp' + exp[0])
    
    plt.show()

C:\Miniconda3\envs\suite2p\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  del sys.path[0]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### concatenated figure

In [126]:
### view neuron sorting

# aligned_stimdata = auxdata[ids['FrameTimes_level'][slice_range][0]:ids['FrameTimes_level'][slice_range][-1],stimops['Stims_chan']]
# aux_time = np.arange(0, aligned_stimdata.shape[0] * 1 / aux_samplingfreq, 1 / aux_samplingfreq  ) - prestim

fs = framerate_auxderived / level
vmin_glob = -.5
vmax_glob = 2.5

fs = framerate_auxderived / level

with sns.axes_style('white'):
    xl = [-prestim,Sm_all.shape[1]/fs - prestim]
    
    fig9 = plt.figure(figsize=(20,12), constrained_layout=True)
    
    gs1 = fig9.add_gridspec(nrows=8, ncols=3)
    
    fig9.add_subplot(gs1[1:, :])      
    fig9_ax=plt.imshow(Sm_all, vmin=vmin_glob,vmax=vmax_glob,aspect='auto',extent=[-prestim,Sm_all.shape[1]/fs-prestim, 0,Sm_all.shape[0]], cmap = 'gray_r')
    plt.xlabel('time (s)', fontsize=18)    
    
#     plt.vlines([prestim_frames / fs-prestim, maxlength / fs - poststim_frames / fs -prestim ],0,ncells-1, 'k')
    plt.ylabel('boutons', fontsize=18)     
    
    plt.vlines([aux_time_full[-1],  +aux_time_full[-1] *2 + poststim],0,ncells-1, 'r')
    plt.vlines([prestim_frames / fs-prestim, maxlength / fs - poststim_frames / fs -prestim],0,ncells-1, 'b')
    
    plt.vlines([aux_time_full[-1] + (prestim_frames / fs),  aux_time_full[-1] + (maxlength / fs - poststim_frames / fs ) ],0,ncells-1, 'b')
    plt.vlines([2 * aux_time_full[-1] + (prestim_frames / fs) + prestim,  2 * aux_time_full[-1] + (maxlength / fs - poststim_frames / fs ) + prestim],0,ncells-1, 'b')
#     plt.plot(aux_time_full_concat, aligned_stimdata_full_concat, 'k')
    
#     plt.colorbar(label='deconvolved F [zscored]')
#    plt.colorbar(label='deconvolved F [a.u.]')
   
    f9_ax4 = fig9.add_subplot(gs1[:1, :])    
    plt.plot(aux_time_full_concat, aligned_stimdata_full_concat, 'k')
#     plt.vlines([prestim_frames / fs-prestim, maxlength / fs - poststim_frames / fs -prestim],0,4, 'k')
    f9_ax4.set_xlim(*xl)
    f9_ax4.axis('off')
#     plt.title('Contra                                    Ipsi                                    Bino')
    plt.text(aux_time_full[-1] / 2 - 2, 5 , 'Contra', fontsize=16)
    plt.text(aux_time_full[-1] * 1.5-2, 5 , 'Ipsi', fontsize=16)
    plt.text(aux_time_full[-1] * 2.5-2, 5 , 'Bino', fontsize=16)
    
   
    
#     plt.tight_layout()           
    plt.suptitle('z-scored deconvolved F - Chirp PSTHs - rastermap embedding (smoothened over boutons) - exp' + exp[0])
    
    plt.show()

C:\Miniconda3\envs\suite2p\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  from ipykernel import kernelapp as app


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# NMF on PSTHs

In [139]:
nmf = NMF(n_components=6, solver="mu")

sp_all[sp_all < 0] = 0
sp_contra[sp_contra < 0] = 0

W = nmf.fit_transform(sp_contra.T)
 
H = nmf.components_

In [89]:
comp_df_H = pd.DataFrame(H.T, columns=['comp1','comp2','comp3','comp4','comp5'])

In [137]:
H

array([[4.33900714e-01, 5.09267209e-01, 2.81470351e-01, ...,
        9.51526165e-02, 8.09387840e-02, 2.17149419e-02],
       [0.00000000e+00, 0.00000000e+00, 1.49837559e-01, ...,
        2.75389473e-02, 2.29323072e-01, 2.62059479e-03],
       [1.48121907e-01, 9.61150413e-02, 0.00000000e+00, ...,
        1.08994136e+00, 4.25127806e-01, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        4.58766197e-01, 2.44759107e-01, 8.65663687e+00],
       [3.52098551e-01, 1.87589184e-01, 3.32075663e-01, ...,
        0.00000000e+00, 2.77387617e-02, 9.19176164e-04],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        2.36265261e-01, 3.39467438e-01, 0.00000000e+00]])

In [140]:
fig, axs = plt.subplots(2,3, figsize=(15, 6), facecolor='w', edgecolor='k')
# fig.subplots_adjust(hspace = .5, wspace=.001)

k = 0

for ax, d in zip(axs.ravel(), H):
    k += 1
    ax.plot(time, d)
    ax.set_title('component' + str(k))
    ax.vlines([prestim_frames / fs-prestim, maxlength / fs - poststim_frames / fs -prestim],0,4, 'k')
# plt.figure() 
# for x in 
# sns.heatmap(data = comp_df_H.T)
plt.show()

C:\Miniconda3\envs\suite2p\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Rastermap embedding exploration

In [ ]:
# model = rm.mapping.Rastermap(n_components=2, n_X=40,  nPC=400, init='pca')
embedding = model_contra.fit_transform(sp_contra.T)

In [ ]:
np.round(embedding*40)/4

In [ ]:
embedding[isort]


In [ ]:
S = sp_contra.T

NN, NT = S.shape 

# reshape and sum S across neurons to get "components"
nc = 100
NC = int(np.floor(NN / nc))
Sp = np.reshape(S[isort][:nc*NC], (NC, nc, NT))
Sp = Sp.mean(axis=1)

In [ ]:
Sp = np.reshape(S[isort][:nc*NC], (NC, nc, NT))



In [ ]:
S[isort][:nc*NC].shape

In [ ]:
# plot results here
cmap = plt.get_cmap('gist_ncar')
cmap = cmap(np.linspace(0,.97,32))
cmap = cmap[np.random.permutation(32),:]
plt.figure(figsize=(6,6))
# each point is colored based on stimulus identity
istim = np.ones(len(sp_ipsi.T), dtype = 'int')
plt.scatter(out2[:,0],out2[:,1],color=cmap[istim,:],marker='x')
plt.show()

In [ ]:
cmap[istim,:]


In [ ]:
plt.figure()
plt.plot(melted_aux[[:,0]], melted_aux[[0,1]])

In [ ]:
melted_aux.shape

### Panda convert to long form

In [ ]:
melted = dataframe.melt(id_vars=['category', 'time', 'trial', 'cell'], value_vars=['aligned_F'])

In [ ]:
melted_aux

In [ ]:
### generating trial numpy array from single cell and condition

arr = melted.loc[(melted.loc[:,'trial'] < 8) & (melted.loc[:,'cell'] == 1) & (melted.loc[:,'category'] == 'Bino'),'value'].to_numpy()
arr = arr.reshape(8,maxlength)


### Panda slicing examples

In [ ]:
### plot single cell including individual trials and means - seaborn and other

# generating trial dataframe from single cell and condtion - SINGLE TRIAL
subselected_orig = melted.loc[(melted.loc[:,'trial'] == 1) & (melted.loc[:,'cell'] == 1) & (melted.loc[:,'category'] == 'Contra'),['time', 'value', 'trial']]

# generating trial dataframe from single cell and condtion - ALL TRIALS
subselected_orig = melted.loc[(melted.loc[:,'cell'] == 1) & (melted.loc[:,'category'] == 'Contra'),['time', 'value', 'trial']]

# reshape ('pivot') to index over time and have single trial columns
subselected = subselected_orig.pivot(index = 'time', columns='trial', values='value')

# reshape ('pivot') to index over time and have single trial columns - zscore on the fly
# subselected = subselected_orig.pivot(index = 'time', columns='trial', values='value').transform((lambda x : zscore(x)))

subselected = (subselected - subselected.mean())/subselected.std(ddof=0)

# generate mean from that

subselected_mean = subselected.mean(axis = 1)

# plot seaborn original frame and pivoted mean and trials next to eachother
fig = plt.figure()
plt.subplot(1,3,1)
sns.lineplot(x = 'time', y = 'value', data = subselected, ci = 'sd')
plt.subplot(1,3,2)
plt.plot(subselected, 'k')
plt.plot(subselected_mean, 'r')
plt.subplot(1,3,3)
# sns.heatmap(subselected, cmap = 'cubehelix')
plt.imshow(subselected.T, aspect = 'auto',  cmap = 'cubehelix')

In [ ]:
subselected = subselected_orig.pivot(index = 'time', columns='trial', values='value')
subselected = (subselected - subselected.mean())/subselected.std(ddof=0)

In [ ]:
### plot all trials of a single cell concatenated

fig = plt.figure()
melted.loc[(melted.loc[:,'cell'] == 1) & (melted.loc[:,'category'] == 'Contra'),'value'].plot()

In [ ]:
fig = plt.figure()
sns.set()
with sns.axes_style('white'):
#     plt.imshow(arr, aspect = 'auto',  cmap = 'cubehelix')
    sns.heatmap(subselected_orig, cmap = 'cubehelix')
    plt.show()

In [ ]:
### plot individual cells as line plots and group by condition

g = sns.FacetGrid(melted, col='category', hue='category', row='cell', sharey='row', margin_titles=True)
g.map(sns.lineplot, 'time', 'value', ci='sd')

In [ ]:
# plt.clf()
g = sns.relplot(x="time", y="value",
                 col="category", row = 'cell', hue="category", style="category",
                 kind="line", data=melted, estimator=None, facet_kws={'sharey': True, 'sharex': True})

### arrays and classic plotting

### plot examples

In [ ]:
# show cells
im = np.zeros((ops['Ly'], ops['Lx']))
ncells = len(stat)

for n in range(0,ncells):
    ypix = stat[n]['ypix'][~stat[n]['overlap']]
    xpix = stat[n]['xpix'][~stat[n]['overlap']]
    im[ypix,xpix] = n+1
fig = plt.figure(figsize=(7,4))
plt.imshow(im)
plt.tight_layout()
plt.show()

In [ ]:
ops['tau'] = .7

In [ ]:
# show mean image
fig = plt.figure(figsize=(8,3))
plt.subplot(1,3,1)
plt.imshow(ops["meanImg"])
plt.subplot(1,3,2)
plt.imshow(ops["meanImgE"])
plt.subplot(1,3,3)
plt.imshow(ops["Vcorr"])
plt.show

In [ ]:
ops['tau'] = 0.5
bouton = 11;

In [ ]:
efilt = np.exp(- np.linspace(0,50,200) / (ops['tau'] * ops['fs']))
#efilt /= efilt.sum()
sout = convolve(spks[bouton,:], efilt)
sout = sout[:spks.shape[1]]

In [ ]:
fig = plt.figure(figsize=(16,4))
plt.plot(F[bouton]-Fneu[bouton] * .7)
plt.show

In [ ]:
plt.plot(sout)
plt.tight_layout()
plt.show
#plt.plot(F[10:])

In [ ]:
plt.plot(spks[11])
plt.show

In [ ]:
plt.figure(figsize=(7,4))
plt.imshow(spks[:100, :5000], vmax = 3, vmin = -0.5, aspect='auto', cmap = 'gray_r')
plt.title('sample of the neural data matrix')
plt.ylabel('boutons') 
plt.xlabel('time [samples]')

In [ ]:
ops

# ToDo

- batch run different deconvolution settings using this code snippet

In [ ]:
# compute deconvolution
from suite2p import dcnv
import numpy as np

tau = 1.0 # timescale of indicator
fs = 30.0 # sampling rate in Hz
neucoeff = 0.7 # neuropil coefficient
# for computing and subtracting baseline
baseline = 'maximin' # take the running max of the running min after smoothing with gaussian
sig_baseline = 10.0 # in bins, standard deviation of gaussian with which to smooth
win_baseline = 60.0 # in seconds, window in which to compute max/min filters

ops = {'tau': tau, 'fs': fs, 'neucoeff': neucoeff,
       'baseline': baseline, 'sig_baseline': sig_baseline, 'win_baseline': win_baseline}

# load traces and subtract neuropil
F = np.load('F.npy')
Fneu = np.load('Fneu.npy')
Fc = F - ops['neucoeff'] * Fneu

# baseline operation
Fc = dcnv.preprocess(Fc, ops)

# get spikes
spks = dcnv.oasis(Fc, ops)